## Publish a Domino Model Endpoint

Prepare for Model Deployment. You can deploy the file `kafka_model.py`. Deploying the model requires the following:-
1. Environment Id - You can get a list of environments by invoking `list_envs` function below
2. Python file (`kafka_model.py` in our case)
3. Function in the Python file  (`predict` in out case)
4. Any additional environment variables the model needs to configure itself. We pass the following:
   - KAFKA_BOOTSTRAP_SERVERS = 'KAFKA_BOOTSTRAP_SERVERS'
   - KAFKA_USER_NAME = 'KAFKA_USERNAME'
   - KAFKA_PASSWORD = 'KAFKA_PASSWORD'
   - FEATURE_TOPIC = 'FEATURE_TOPIC' 
   - INFERENCE_GROUP_ID = 'INFERENCE_GROUP_ID' (A model can have many instances to serve, each of them should use the same Consumer Group Id to ensure a Kafka message is usually processed only once)
   - PREDICTION_TOPIC = Topic where predictions are written




In [5]:
from domino import Domino
import os
import time
api_host = os.getenv("DOMINO_API_HOST")
project_id=os.environ['DOMINO_PROJECT_ID']
dm_api = Domino(
        "{}/{}".format(
            os.getenv("DOMINO_PROJECT_OWNER"), os.getenv("DOMINO_PROJECT_NAME")
        )
    )

def list_envs():
    all_available_environments = dm_api.environments_list()
    global_environments = list(
        filter(
            lambda x: x.get("visibility") == "Global",
            all_available_environments["data"],
        )
    )
    return all_available_environments["data"]

# nv_pairs is a dictionary of name/value pairs, {'name': 'value'}
def add_env_vars(model_id, nv_pairs):
    vars_array = [{"name": name, "value": value} for (name, value) in nv_pairs.items()]
    request = {"vars": vars_array}
    api_host = os.getenv("DOMINO_API_HOST")
    resp = dm_api.request_manager.post(
        f"{api_host}/models/{model_id}/environmentVariables/add", json=request
    )


Get a list of all environments

```
[{'id': '63fa71e8e05a501c43a61ac6', 'name': 'KAFKA', 'visibility': 'Private'},
 {'id': '63f7c9e768e90a0b79d68d16',
  'name': '5.5 Snowflake Model Export Environment (Domino Internal)',
  'visibility': 'Global'},
 {'id': '63f7c9e768e90a0b79d68d13',
  'name': '5.5 Spark Compute Environment',
  'visibility': 'Global'},
 {'id': '63f7c9e768e90a0b79d68d12',
  'name': '5.5 Spark Cluster Environment',
  'visibility': 'Global'},
 {'id': '63f7c9e768e90a0b79d68d11',
  'name': 'Domino Standard Environment Py3.9 R4.2',
  'visibility': 'Global'}]
```

In [6]:
list_envs()

[{'id': '64231086f4ae3e505ae9f76e',
  'name': 'online-learning',
  'visibility': 'Private'},
 {'id': '64230b0df9b0ca482d83a6fd',
  'name': '5.5 Snowflake Model Export Environment (Domino Internal)',
  'visibility': 'Global'},
 {'id': '64230b0df9b0ca482d83a6fa',
  'name': '5.5 Spark Compute Environment',
  'visibility': 'Global'},
 {'id': '64230b0df9b0ca482d83a6f9',
  'name': '5.5 Spark Cluster Environment',
  'visibility': 'Global'},
 {'id': '64230b0df9b0ca482d83a6f8',
  'name': 'Domino Standard Environment Py3.9 R4.2',
  'visibility': 'Global'}]

In [7]:

import os
env_id='64231086f4ae3e505ae9f76e'

model_file='src/model.py'
model_function='predict'


KAFKA_BOOTSTRAP_SERVERS = 'KAFKA_BOOTSTRAP_SERVERS'
KAFKA_USERNAME = 'KAFKA_USER_NAME'
KAFKA_PASSWORD = 'KAFKA_PASSWORD'
FEATURE_TOPIC = 'FEATURE_TOPIC'
INFERENCE_GROUP_ID = 'INFERENCE_GROUP_ID'
PREDICTION_TOPIC='PREDICTION_TOPIC'
PREDICTION_TOPIC='PREDICTION_TOPIC'
MODEL_UPDATE_TOPIC='MODEL_UPDATE_TOPIC'
#model_name= os.environ['DOMINO_PROJECT_NAME'] 
model_name= 'icde23'
env_variables = {
                 KAFKA_BOOTSTRAP_SERVERS:os.environ[KAFKA_BOOTSTRAP_SERVERS],
                 KAFKA_USERNAME:os.environ[KAFKA_USERNAME],
                 KAFKA_PASSWORD:os.environ[KAFKA_PASSWORD],
                 FEATURE_TOPIC: 'features',
                 INFERENCE_GROUP_ID: f'{model_name}-03-27-10PM',
                 PREDICTION_TOPIC: 'predictions',
                 MODEL_UPDATE_TOPIC: 'model_updates'
                }
 

## Publish Model

In [8]:
published_model = dm_api.model_publish(
        file=model_file,
        function=model_function,
        environment_id=env_id,
        name=f'{model_name}-v1',
        description="Autopublish of MLFLOW model {}".format(model_name),
    )
published_model_id = published_model.get("data", {}).get("_id")
print(f'Published Model{published_model_id}')
print('Now add env variables')

add_env_vars(published_model_id,env_variables)
resp = dm_api.request_manager.get(f'{api_host}/models/{published_model_id}/activeStatus',json={})

status = ''
if resp.status_code==200:
    status = resp.json()['status']
print(status)
resp = dm_api.request_manager.get(
        f"{api_host}/models/{published_model_id}/activeStatus", json={}
    )
while resp.json()['isPending']:
    print('Sleeping for 30 seconds')
    time.sleep(30)
    
    resp = dm_api.request_manager.get(
        f"{api_host}/models/{published_model_id}/activeStatus", json={}
    )
    print(resp.json())
status = resp.json()['status']
print(status)

Published Model642332d9b5706f3fda6211db
Now add env variables
Stopped
Sleeping for 30 seconds
{'modelId': {'value': '642332d9b5706f3fda6211db'}, 'modelVersionId': {'value': '642332d9b5706f3fda6211dd'}, 'status': 'Preparing to build', 'operations': [{'startTime': 1680028377993, 'lastUpdated': 1680028378593, 'sagaDescription': 'Deploy model version', 'shortStateDescription': 'Building', 'longStateDescription': 'Image Building', 'isFailure': False}], 'lastOperation': {'startTime': 1680028377993, 'lastUpdated': 1680028378593, 'sagaDescription': 'Deploy model version', 'shortStateDescription': 'Building', 'longStateDescription': 'Image Building', 'isFailure': False}, 'isPending': True}
Sleeping for 30 seconds
{'modelId': {'value': '642332d9b5706f3fda6211db'}, 'modelVersionId': {'value': '642332d9b5706f3fda6211dd'}, 'status': 'Preparing to build', 'operations': [{'startTime': 1680028377993, 'lastUpdated': 1680028378593, 'sagaDescription': 'Deploy model version', 'shortStateDescription': 'Bui